In [1]:

!apt-get -y install ffmpeg
!pip install --quiet pydub tqdm pandas
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
import tarfile, pandas as pd
from pydub import AudioSegment, silence
from tqdm import tqdm



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Mounted at /content/drive


In [ ]:
for m in tarfile.open(ARCHIVE_PATH, "r:gz").getmembers():
    if m.name.endswith("sw/validated.tsv") or m.name.endswith(".mp3"):
      print(m.name)

In [11]:
# prompt: when i print df, it shows empty. can you modify the code and fix it?

ARCHIVE_PATH = '/content/drive/MyDrive/cv-corpus-14.0-delta-2023-06-23-sw.tar.gz' # Replace with your actual archive path

data = []
with tarfile.open(ARCHIVE_PATH, "r:gz") as tar:
    for m in tar.getmembers():
        if m.name.endswith("sw/validated.tsv"):
            print(f"Processing {m.name}")
            f = tar.extractfile(m)
            if f:
                # Decode bytes to string before passing to StringIO for pandas
                import io
                data = pd.read_csv(io.StringIO(f.read().decode('utf-8')), sep='\t', header=0)
                break # Assuming only one validated.tsv file
df = pd.DataFrame(data)
df


Processing cv-corpus-14.0-delta-2023-06-23/sw/validated.tsv


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,0460dfd5fa87ebc875217a43036befcd14f0ffbaf327ee...,common_voice_sw_37664539.mp3,Ugongwa wa kupinda shingo,2,0,twenties,male,NaN,NaN,sw,NaN
1,1abea1658e3aae2ec2cf50d27928ef504cacd9cb7cf9c4...,common_voice_sw_37860685.mp3,Dalili ya ugonjwa wa miguu na midomo ni mnyama...,21,1,twenties,male,NaN,Kingwana (DRC),sw,NaN
2,1fd73ffae606e10a0b84b76463f0e99583d4524e67bfee...,common_voice_sw_37772967.mp3,Wanyama walioathirika na ugonjwa wa ukurutu wa...,12,0,NaN,NaN,NaN,NaN,sw,NaN
3,2b3cad939e1679fa6dbc2135a4f4df92f258c1e6c51d90...,common_voice_sw_37911549.mp3,Hata hivyo maelfu ya watu waliojitokeza kumsik...,2,0,twenties,male,NaN,NaN,sw,NaN
4,3837276491d1b8399ca8f4be181c4a9ea0aa2a85e25f4a...,common_voice_sw_37806408.mp3,Marais Uhuru Kenyata wa Kenya Yoweri Museveni ...,3,0,NaN,NaN,NaN,NaN,sw,NaN
...,...,...,...,...,...,...,...,...,...,...,...
265,9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...,common_voice_sw_37982384.mp3,"Lakini, kwa mujibu wa ripoti ya Benki ya Dunia...",2,0,twenties,male,kenyan accent,Kiswahili Sanifu (EA),sw,NaN
266,9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...,common_voice_sw_37982400.mp3,Binti ya Dos Santos apinga kuzikwa kwake Angola,2,0,twenties,male,kenyan accent,Kiswahili Sanifu (EA),sw,NaN
267,9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...,common_voice_sw_37982408.mp3,Ripoti ya Baraza la Jumuiya ya Afrika Masharik...,2,0,twenties,male,kenyan accent,Kiswahili Sanifu (EA),sw,NaN
268,9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...,common_voice_sw_37982412.mp3,Toa taarifa ukishuku kuwepo kwa taarifa yoyote...,2,0,twenties,male,kenyan accent,Kiswahili Sanifu (EA),sw,NaN


In [9]:
ARCHIVE_PATH = Path("/content/drive/MyDrive/cv-corpus-14.0-delta-2023-06-23-sw.tar.gz")
EXTRACT_DIR  = Path("common_voice_sw")
OUTPUT_WAVS  = Path("cleaned_sw_audio")
MANIFEST_CSV = Path("manifest_sw.csv")
TARGET_SR    = 16_000
SIL_THRESH   = -40
MIN_MS       = 200

EXTRACT_DIR.mkdir(exist_ok=True)
OUTPUT_WAVS.mkdir(exist_ok=True)

# get tsv/clips
with tarfile.open(ARCHIVE_PATH, "r:gz") as tar:
    members = [m for m in tar.getmembers() if m.name.endswith("sw/validated.tsv") or m.name.endswith(".mp3")]
    tar.extractall(path=EXTRACT_DIR, members=members)

tsv_path = next(EXTRACT_DIR.rglob("validated.tsv"))
df = pd.read_csv(tsv_path, sep="\t")
print(df)

                                             client_id  \
0    0460dfd5fa87ebc875217a43036befcd14f0ffbaf327ee...   
1    1abea1658e3aae2ec2cf50d27928ef504cacd9cb7cf9c4...   
2    1fd73ffae606e10a0b84b76463f0e99583d4524e67bfee...   
3    2b3cad939e1679fa6dbc2135a4f4df92f258c1e6c51d90...   
4    3837276491d1b8399ca8f4be181c4a9ea0aa2a85e25f4a...   
..                                                 ...   
265  9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...   
266  9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...   
267  9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...   
268  9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...   
269  9f000f726aa0ffbce77c5a228041dfc7afac116e634aa0...   

                             path  \
0    common_voice_sw_37664539.mp3   
1    common_voice_sw_37860685.mp3   
2    common_voice_sw_37772967.mp3   
3    common_voice_sw_37911549.mp3   
4    common_voice_sw_37806408.mp3   
..                            ...   
265  common_voice_sw_37982384.mp3   
266  co

In [6]:


rows = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="processing audio"):
    mp3_abs = EXTRACT_DIR / f"cv-corpus-14.0-delta-2023-06-23/sw/clips/{row['path']}"
    if not mp3_abs.exists():
        continue

    try:
        audio = AudioSegment.from_file(mp3_abs, format="mp3")
    except Exception as e:
        print(" invalid file:", mp3_abs.name, e)
        continue

    #trim leading/silence
    chunks = silence.split_on_silence(audio, silence_thresh=SIL_THRESH, min_silence_len=150)
    if chunks:
        audio = sum(chunks)

    if len(audio) < MIN_MS:
        continue

    #resample
    audio = audio.set_frame_rate(TARGET_SR).set_channels(1)

    #export
    wav_name = row["path"].replace(".mp3", ".wav")
    wav_out  = OUTPUT_WAVS / wav_name
    wav_out.parent.mkdir(parents=True, exist_ok=True)
    audio.export(wav_out, format="wav")

    rows.append({
        "wav_path"  : str(wav_out.resolve()),
        "duration"  : len(audio) / 1000.0,
        "transcript": row["sentence"]
    })

#manifest
pd.DataFrame(rows).to_csv(MANIFEST_CSV, index=False)
print(f"\n kept {len(rows):,} clips.")
print("   wavs: ", OUTPUT_WAVS.resolve())
print("   csv: ", MANIFEST_CSV.resolve())


processing audio: 100%|██████████| 270/270 [01:23<00:00,  3.23it/s]


 kept 270 clips.
   wavs:  /content/cleaned_sw_audio
   csv:  /content/manifest_sw.csv
